In [1085]:
#load the library
import numpy as np
import pandas as pd

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_iris 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import optuna
import warnings
from tqdm import tqdm
import time
from sklearn.decomposition import TruncatedSVD, PCA
import joblib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [1086]:
#import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('gender_submission.csv')

In [1087]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [1088]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [1089]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [1090]:
train.shape

(891, 12)

In [1091]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Fill NAs

In [1092]:
sub.PassengerId.to_list() == test.PassengerId.to_list()

True

In [1093]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [1094]:
## fill age
# train.Age = train.Age.fillna(np.mean(train.Age))
train.Age = train.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))

In [1095]:
train.Cabin = train.Cabin.fillna('Unknown')

In [1096]:
train.Embarked = train.Embarked.fillna('Unknown')

In [1097]:
train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [1098]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [1099]:
# test.Age = test.Age.fillna(np.median(train.Age))
test.Age = test.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
test.Fare = test.Fare.fillna(np.median(test.Fare))

In [1100]:
test.Cabin = test.Cabin.fillna('Unknown')

In [1101]:
test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin          0
Embarked       0
dtype: int64

## Split folds

In [1102]:
#do k fold validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [1103]:
train['fold'] = -999
for fold_id, (train_index, valid_index) in enumerate(skf.split(X=train, y=train.Survived.values)):
    train.loc[valid_index, 'fold'] = fold_id

In [1104]:
train.fold.value_counts()

0    179
1    178
2    178
3    178
4    178
Name: fold, dtype: int64

## Feature Engineering

#### 1. Name Title (Change the name to title and label encoding)

In [1105]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [1106]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [1107]:
test['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(test['Title'], test['Sex'])

Sex,female,male
Title,,
Col,0,2
Dona,1,0
Dr,0,1
Master,0,21
Miss,78,0
Mr,0,240
Mrs,72,0
Ms,1,0
Rev,0,2


In [1108]:
test['Title'] = test['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')

In [1109]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

train['Title'] = train['Title'].map(title_mapping)
train['Title'] = train['Title'].fillna(0)
test['Title'] = test['Title'].map(title_mapping)
test['Title'] = test['Title'].fillna(0)

In [1110]:
# train = train.drop('Name', axis=1)
# test = test.drop('Name', axis=1)

In [1111]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,0,1


In [1112]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,Unknown,S,3


### 2.  Cut the Age and Fare column to different bins 

In [1113]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,fold,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.112424,0.523008,0.381594,32.204208,1.997755,1.728395
std,257.353842,0.486592,0.836071,13.304424,1.102743,0.806057,49.693429,1.415800,1.030039
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,21.500000,0.000000,0.000000,7.910400,1.000000,1.000000
50%,446.000000,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200,2.000000,1.000000
75%,668.500000,1.000000,3.000000,36.000000,1.000000,0.000000,31.000000,3.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,4.000000,5.000000


In [1114]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Title
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000,418.000000
mean,1100.500000,2.265550,29.297847,0.447368,0.392344,35.627188,1.741627
std,120.810458,0.841838,13.038856,0.896760,0.981429,55.907576,1.006091
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,996.250000,1.000000,22.000000,0.000000,0.000000,7.895800,1.000000
50%,1100.500000,3.000000,25.000000,0.000000,0.000000,14.454200,1.000000
75%,1204.750000,3.000000,36.375000,1.000000,0.000000,31.500000,2.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,5.000000


In [1115]:
train['Fare'] = pd.qcut(train['Fare'], 10)
test['Fare'] = pd.qcut(test['Fare'], 10)

In [1116]:
train['Fare'].value_counts()

(7.854, 8.05]        106
(-0.001, 7.55]        92
(27.0, 39.688]        91
(21.679, 27.0]        89
(39.688, 77.958]      89
(14.454, 21.679]      88
(7.55, 7.854]         87
(77.958, 512.329]     87
(10.5, 14.454]        84
(8.05, 10.5]          78
Name: Fare, dtype: int64

In [1117]:
train['Age'].value_counts()

25.00    117
21.50     42
40.00     34
30.00     34
24.00     30
        ... 
80.00      1
0.42       1
34.50      1
66.00      1
12.00      1
Name: Age, Length: 89, dtype: int64

In [1118]:
train['Age'] = pd.qcut(train['Age'], 10)
test['Age'] = pd.qcut(test['Age'], 10)

In [1119]:
train['Age'].value_counts()

(22.0, 25.0]     164
(34.0, 40.0]     116
(26.0, 30.0]     101
(0.419, 16.0]    100
(20.0, 22.0]      94
(47.0, 80.0]      89
(16.0, 20.0]      79
(30.0, 34.0]      69
(40.0, 47.0]      61
(25.0, 26.0]      18
Name: Age, dtype: int64

### 3. split sibsp column into different case based on the size

In [1120]:
train['SibSp'].value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [1121]:
test['SibSp'].value_counts()

0    283
1    110
2     14
3      4
4      4
8      2
5      1
Name: SibSp, dtype: int64

In [1122]:
#0 -- small family size; 1 -- middle family size; 2 -- large family size
train['SibSp'].loc[train['SibSp'] < 3] = 0
train['SibSp'].loc[(train['SibSp'] < 5) & (train['SibSp'] >= 3)] = 1
train['SibSp'].loc[train['SibSp'] >= 5] = 2

test['SibSp'].loc[test['SibSp'] < 3] = 0
test['SibSp'].loc[(test['SibSp'] < 5) & (test['SibSp'] >= 3)] = 1
test['SibSp'].loc[test['SibSp'] >= 5] = 2

In [1123]:
train['SibSp'].value_counts()

0    845
1     34
2     12
Name: SibSp, dtype: int64

In [1124]:
test['SibSp'].value_counts()

0    407
1      8
2      3
Name: SibSp, dtype: int64

In [1125]:
# train['Fare'] = str(train['Fare'])
# test['Fare'] = str(test['Fare'])

# train['Age'] = str(train['Age'])
# test['Age'] = str(test['Age'])

# train['SibSp'] = str(train['SibSp'])
# test['SibSp'] = str(test['SibSp'])

In [1126]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,"(20.0, 22.0]",0,0,A/5 21171,"(-0.001, 7.55]",Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,"(34.0, 40.0]",0,0,PC 17599,"(39.688, 77.958]",C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,"(25.0, 26.0]",0,0,STON/O2. 3101282,"(7.854, 8.05]",Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,"(34.0, 40.0]",0,0,113803,"(39.688, 77.958]",C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,"(34.0, 40.0]",0,0,373450,"(7.854, 8.05]",Unknown,S,0,1


In [1127]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,"(33.0, 41.0]",0,0,330911,"(7.796, 8.05]",Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,"(41.0, 48.0]",0,0,363272,"(-0.001, 7.642]",Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,"(48.0, 76.0]",0,0,240276,"(8.05, 11.025]",Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,"(25.0, 28.1]",0,0,315154,"(8.05, 11.025]",Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,"(21.0, 23.0]",0,1,3101298,"(11.025, 14.454]",Unknown,S,3


## Extract Cabin Number from Cabin

In [1128]:
train['Cabin Num'] = train['Cabin'].str.extract(r'([0-9]+)').fillna(0).astype(float)
test['Cabin Num'] = test['Cabin'].str.extract(r'([0-9]+)').fillna(0).astype(float)

In [1129]:
train['Cabin Class'] = train['Cabin'].str.extract(r'([A-Z]+)').fillna(0)
test['Cabin Class'] = test['Cabin'].str.extract(r'([A-Z]+)').fillna(0)

## Label Encoding

In [1130]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title,Cabin Num,Cabin Class
0,1,0,3,"Braund, Mr. Owen Harris",male,"(20.0, 22.0]",0,0,A/5 21171,"(-0.001, 7.55]",Unknown,S,1,1,0.0,U
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,"(34.0, 40.0]",0,0,PC 17599,"(39.688, 77.958]",C85,C,4,3,85.0,C
2,3,1,3,"Heikkinen, Miss. Laina",female,"(25.0, 26.0]",0,0,STON/O2. 3101282,"(7.854, 8.05]",Unknown,S,3,2,0.0,U
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,"(34.0, 40.0]",0,0,113803,"(39.688, 77.958]",C123,S,3,3,123.0,C
4,5,0,3,"Allen, Mr. William Henry",male,"(34.0, 40.0]",0,0,373450,"(7.854, 8.05]",Unknown,S,0,1,0.0,U


In [1131]:
#transfrom categorical data to numbers
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [1132]:
# train.Name = train.Name.str.extract('([A-Za-z]+)\.', expand=False)
# test.Name = test.Name.str.extract('([A-Za-z]+)\.', expand=False)

In [1133]:
train = train.drop('Name', axis=1)
test = test.drop('Name', axis=1)

In [1134]:
train = train.drop('PassengerId', axis=1)
test = test.drop('PassengerId', axis=1)

In [1036]:
train, test = label_encoding(train, test, ['Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', 'Cabin Class', 'Fare'])

Fare


In [1137]:
# pd.get_dummies(train['Fare'])

In [984]:
test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title,Cabin Num,Cabin Class
0,NaN,3,1,14,0,0,376,"(7.796, 8.05]",186,1,NaN,1,0.0,8
1,NaN,3,0,17,0,0,582,"(-0.001, 7.642]",186,2,NaN,3,0.0,8
2,NaN,2,1,19,0,0,184,"(8.05, 11.025]",186,1,NaN,1,0.0,8
3,NaN,3,1,10,0,0,366,"(8.05, 11.025]",186,2,NaN,1,0.0,8
4,NaN,3,0,5,0,1,338,"(11.025, 14.454]",186,2,NaN,3,0.0,8


In [1138]:
# train.describe()

In [146]:
test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title,Cabin Num,Cabin Class
0,NaN,3,1,1,0,0,376,0,186,1,NaN,1,0.0,U
1,NaN,3,0,1,0,0,582,0,186,2,NaN,3,0.0,U
2,NaN,2,1,1,0,0,184,0,186,1,NaN,1,0.0,U
3,NaN,3,1,1,0,0,366,0,186,2,NaN,1,0.0,U
4,NaN,3,0,1,0,1,338,0,186,2,NaN,3,0.0,U


In [1139]:
# test.describe()

# Building Model

## LGB

In [44]:
def optuna_lgb(n_trials=100):
    
    def objective(trial):

        params = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-5, 10, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-5, 10, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 62),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 0.9),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 0.9),
            "bagging_freq": trial.suggest_int("bagging_freq", 2, 9),
            "min_child_samples": trial.suggest_int("min_child_samples", 33, 93),
            "max_depth": trial.suggest_int("max_depth", 2, 7)
        }

        # initialize oof 
        oof_train = np.zeros((len(train),))

        for i in range(5):
            train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
            train_y = train.query(f'fold!={i}').Survived

            valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
            valid_y = train.query(f'fold=={i}').Survived
            
            lgb_train = lgb.Dataset(train_x,
                                    train_y)

            lgb_eval = lgb.Dataset(valid_x,
                                   valid_y,
                                   reference=lgb_train)

            gbm = lgb.train(params,
                            lgb_train,
                            valid_sets=[lgb_train, lgb_eval],
                            num_boost_round=5000,
                            verbose_eval=-1, 
                            early_stopping_rounds=1000
                            )
            oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
            oof_train[valid_x.index] = gbm.predict(valid_x,
                                                   num_iteration=gbm.best_iteration)
            
            
        return accuracy_score(train.Survived, oof_train.round())
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best_params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }
    best_params.update(study.best_params)
    ## save trained model
    # initialize oof 
    oof_train = np.zeros((len(train),))
    print('*'*50)
    print('Exporting best models')
    print('*' * 50)
    # split folds using RSGKF
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        lgb_train = lgb.Dataset(train_x,
                                train_y)

        lgb_eval = lgb.Dataset(valid_x,
                               valid_y,
                               reference=lgb_train)

        gbm = lgb.train(best_params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_eval],
                        num_boost_round=50000,
                        verbose_eval=-1, 
                        early_stopping_rounds=1001
                        )

        oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        oof_train[valid_x.index] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)

        joblib.dump(gbm, f'lgb_fold{i}.pkl')

In [45]:
optuna_lgb() #0.8338945005611672(5 fold)  0.8451178451178452(10 fold)

[I 2021-10-02 20:06:30,993] A new study created in memory with name: no-name-3e6934d0-a518-4e1f-8c05-50467b58a0ce


Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[93]	training's binary_logloss: 0.380899	valid_1's binary_logloss: 0.415107
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[226]	training's binary_logloss: 0.353536	valid_1's binary_logloss: 0.399172
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[131]	training's binary_logloss: 0.359263	valid_1's binary_logloss: 0.458907
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[516]	training's binary_logloss: 0.315719	valid_1's binary_logloss: 0.413806
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:06:32,904] Trial 0 finished with value: 0.8170594837261503 and parameters: {'lambda_l1': 0.4504811473905986, 'lambda_l2': 1.355569361399226, 'num_leaves': 29, 'feature_fraction': 0.5243510836328067, 'bagging_fraction': 0.7409490661146226, 'bagging_freq': 3, 'min_child_samples': 42, 'max_depth': 3}. Best is trial 0 with value: 0.8170594837261503.


Early stopping, best iteration is:
[35]	training's binary_logloss: 0.41142	valid_1's binary_logloss: 0.42258
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[352]	training's binary_logloss: 0.357577	valid_1's binary_logloss: 0.413776
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[579]	training's binary_logloss: 0.336561	valid_1's binary_logloss: 0.389818
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[162]	training's binary_logloss: 0.377119	valid_1's binary_logloss: 0.453438
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[287]	training's binary_logloss: 0.360804	valid_1's binary_logloss: 0.42291
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:06:35,124] Trial 1 finished with value: 0.8159371492704826 and parameters: {'lambda_l1': 0.0029842956381218428, 'lambda_l2': 0.1537110283963997, 'num_leaves': 8, 'feature_fraction': 0.6438285308285523, 'bagging_fraction': 0.5542362596910381, 'bagging_freq': 3, 'min_child_samples': 69, 'max_depth': 5}. Best is trial 0 with value: 0.8170594837261503.


...

Early stopping, best iteration is:
[38]	training's binary_logloss: 0.421855	valid_1's binary_logloss: 0.429076
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[707]	training's binary_logloss: 0.342242	valid_1's binary_logloss: 0.424481
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[446]	training's binary_logloss: 0.363941	valid_1's binary_logloss: 0.39329
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[478]	training's binary_logloss: 0.35264	valid_1's binary_logloss: 0.442948
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[444]	training's binary_logloss: 0.360324	valid_1's binary_logloss: 0.407272
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:10:56,048] Trial 98 finished with value: 0.8215488215488216 and parameters: {'lambda_l1': 0.0005761619417749397, 'lambda_l2': 0.0006495909108405914, 'num_leaves': 19, 'feature_fraction': 0.4935433427693372, 'bagging_fraction': 0.7734503899412688, 'bagging_freq': 6, 'min_child_samples': 66, 'max_depth': 2}. Best is trial 83 with value: 0.8271604938271605.


Early stopping, best iteration is:
[39]	training's binary_logloss: 0.425112	valid_1's binary_logloss: 0.4223
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[402]	training's binary_logloss: 0.347946	valid_1's binary_logloss: 0.418355
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[466]	training's binary_logloss: 0.343894	valid_1's binary_logloss: 0.386765
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[471]	training's binary_logloss: 0.333902	valid_1's binary_logloss: 0.451233
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[444]	training's binary_logloss: 0.340776	valid_1's binary_logloss: 0.411588
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:10:59,401] Trial 99 finished with value: 0.8260381593714927 and parameters: {'lambda_l1': 0.00014732329310346827, 'lambda_l2': 0.008944795514500963, 'num_leaves': 10, 'feature_fraction': 0.45366197239162337, 'bagging_fraction': 0.816636595602557, 'bagging_freq': 6, 'min_child_samples': 71, 'max_depth': 3}. Best is trial 83 with value: 0.8271604938271605.


Early stopping, best iteration is:
[32]	training's binary_logloss: 0.425942	valid_1's binary_logloss: 0.426207
Number of finished trials: 100
Best trial:
  Value: 0.8271604938271605
  Params: 
    lambda_l1: 0.6043319259737905
    lambda_l2: 0.002802289292840519
    num_leaves: 15
    feature_fraction: 0.40515811396792195
    bagging_fraction: 0.8508822113345822
    bagging_freq: 7
    min_child_samples: 74
    max_depth: 2
**************************************************
Exporting best models
**************************************************
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:
[793]	training's binary_logloss: 0.350196	valid_1's binary_logloss: 0.425089
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:
[569]	training's binary_logloss: 0.367938	valid_1's binary_logloss: 0.398122
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:


## XGBoost

In [46]:
#optuna xgboost example
#https://github.com/optuna/optuna-examples/blob/main/xgboost/xgboost_integration.py
import xgboost as xgb
def objective(trial):
    param = {
            "verbosity": 0,
            "objective": "binary:logistic",  #binary:logistic
            "eval_metric": "auc",  #map
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        
        accuracy = []
        # Add a callback for pruning.
        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
        bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
        preds = bst.predict(dvalid)
        pred_labels = np.rint(preds)
        accuracy.append(sklearn.metrics.accuracy_score(valid_y, pred_labels))
        
        
        joblib.dump(bst, f'xgboost_fold{i}.pkl')
    return np.mean(accuracy)

    
if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

[I 2021-10-02 20:11:03,624] A new study created in memory with name: no-name-2aee924a-86d7-41db-86c7-4d3108934966


[0]	validation-auc:0.50000
[1]	validation-auc:0.50000
[2]	validation-auc:0.50000
[3]	validation-auc:0.50000
[4]	validation-auc:0.50000
[5]	validation-auc:0.50000
[6]	validation-auc:0.77582
[7]	validation-auc:0.77582
[8]	validation-auc:0.77582
[9]	validation-auc:0.77582
[0]	validation-auc:0.50000
[1]	validation-auc:0.50000
[2]	validation-auc:0.50000
[3]	validation-auc:0.50000
[4]	validation-auc:0.50000
[5]	validation-auc:0.74799
[6]	validation-auc:0.74799
[7]	validation-auc:0.74799
[8]	validation-auc:0.74799
[9]	validation-auc:0.74799
[0]	validation-auc:0.50000
[1]	validation-auc:0.50000
[2]	validation-auc:0.50000
[3]	validation-auc:0.50000
[4]	validation-auc:0.50000
[5]	validation-auc:0.50000
[6]	validation-auc:0.75468
[7]	validation-auc:0.75468
[8]	validation-auc:0.75468
[9]	validation-auc:0.75468
[0]	validation-auc:0.50000
[1]	validation-auc:0.50000
[2]	validation-auc:0.50000
[3]	validation-auc:0.50000
[4]	validation-auc:0.75013
[5]	validation-auc:0.75013
[6]	validation-auc:0.75013
[

[I 2021-10-02 20:11:03,895] Trial 0 finished with value: 0.8033707865168539 and parameters: {'booster': 'dart', 'lambda': 0.0002570549706052807, 'alpha': 0.02030012135188256, 'max_depth': 1, 'eta': 1.7764453527041365e-08, 'gamma': 0.007775901187686644, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0002613272964360827, 'skip_drop': 0.0016512664981552646}. Best is trial 0 with value: 0.8033707865168539.


[0]	validation-auc:0.84908
[1]	validation-auc:0.84908
[2]	validation-auc:0.84756
[3]	validation-auc:0.84914
[4]	validation-auc:0.84862
[5]	validation-auc:0.84578
[6]	validation-auc:0.84223
[7]	validation-auc:0.84249
[8]	validation-auc:0.84684
[9]	validation-auc:0.84618
[0]	validation-auc:0.83315
[1]	validation-auc:0.84626
[2]	validation-auc:0.84318
[3]	validation-auc:0.84211
[4]	validation-auc:0.84158
[5]	validation-auc:0.84318
[6]	validation-auc:0.84332
[7]	validation-auc:0.84291
[8]	validation-auc:0.84278
[9]	validation-auc:0.84265
[0]	validation-auc:0.81544
[1]	validation-auc:0.81457
[2]	validation-auc:0.81771
[3]	validation-auc:0.81691
[4]	validation-auc:0.81691
[5]	validation-auc:0.81350
[6]	validation-auc:0.81310
[7]	validation-auc:0.81143
[8]	validation-auc:0.81290
[9]	validation-auc:0.81370
[0]	validation-auc:0.84432
[1]	validation-auc:0.84057
[2]	validation-auc:0.82640
[3]	validation-auc:0.82447
[4]	validation-auc:0.83382
[5]	validation-auc:0.83315
[6]	validation-auc:0.83422
[

[I 2021-10-02 20:11:04,332] Trial 1 finished with value: 0.8146067415730337 and parameters: {'booster': 'dart', 'lambda': 8.437116712211345e-06, 'alpha': 0.00047727449067108663, 'max_depth': 9, 'eta': 0.010950398930002736, 'gamma': 1.0901304038393135e-08, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.002255729522844026, 'skip_drop': 4.884696429130731e-07}. Best is trial 1 with value: 0.8146067415730337.


...

[0]	validation-auc:0.85237
[1]	validation-auc:0.87128
[2]	validation-auc:0.87714
[3]	validation-auc:0.88900
[4]	validation-auc:0.88551
[5]	validation-auc:0.87563
[6]	validation-auc:0.87760
[7]	validation-auc:0.87635
[8]	validation-auc:0.87161
[9]	validation-auc:0.87780
[0]	validation-auc:0.84098
[1]	validation-auc:0.86684
[2]	validation-auc:0.85214
[3]	validation-auc:0.86310
[4]	validation-auc:0.86972
[5]	validation-auc:0.87139
[6]	validation-auc:0.85588
[7]	validation-auc:0.85528
[8]	validation-auc:0.85762
[9]	validation-auc:0.85829
[0]	validation-auc:0.82059
[1]	validation-auc:0.83001
[2]	validation-auc:0.84913
[3]	validation-auc:0.84298
[4]	validation-auc:0.83777
[5]	validation-auc:0.84238
[6]	validation-auc:0.82747
[7]	validation-auc:0.83102
[8]	validation-auc:0.83476
[9]	validation-auc:0.83483
[0]	validation-auc:0.85227
[1]	validation-auc:0.84592
[2]	validation-auc:0.85488
[3]	validation-auc:0.85702
[4]	validation-auc:0.85582
[5]	validation-auc:0.85528
[6]	validation-auc:0.85154
[

[I 2021-10-02 20:11:28,684] Trial 98 finished with value: 0.8146067415730337 and parameters: {'booster': 'dart', 'lambda': 1.0587471280538744e-08, 'alpha': 1.431410613038094e-05, 'max_depth': 7, 'eta': 0.6108640924800371, 'gamma': 5.60408954949805e-07, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 9.920647500199648e-07, 'skip_drop': 0.1503460791781096}. Best is trial 58 with value: 0.8370786516853933.


[0]	validation-auc:0.87787
[1]	validation-auc:0.88880
[2]	validation-auc:0.88518
[3]	validation-auc:0.88689
[4]	validation-auc:0.88307
[5]	validation-auc:0.88491
[6]	validation-auc:0.88584
[7]	validation-auc:0.88544
[8]	validation-auc:0.88610
[9]	validation-auc:0.88505
[0]	validation-auc:0.84352
[1]	validation-auc:0.84352
[2]	validation-auc:0.84352
[3]	validation-auc:0.84352
[4]	validation-auc:0.84352
[5]	validation-auc:0.84311
[6]	validation-auc:0.84338
[7]	validation-auc:0.84338
[8]	validation-auc:0.86203
[9]	validation-auc:0.86283
[0]	validation-auc:0.83576
[1]	validation-auc:0.83576
[2]	validation-auc:0.83576
[3]	validation-auc:0.83576
[4]	validation-auc:0.83576
[5]	validation-auc:0.83576
[6]	validation-auc:0.83696
[7]	validation-auc:0.83696
[8]	validation-auc:0.83696
[9]	validation-auc:0.84138
[0]	validation-auc:0.83750
[1]	validation-auc:0.84044
[2]	validation-auc:0.83817
[3]	validation-auc:0.84766
[4]	validation-auc:0.84779
[5]	validation-auc:0.85214
[6]	validation-auc:0.85281
[

[I 2021-10-02 20:11:29,061] Trial 99 finished with value: 0.8370786516853933 and parameters: {'booster': 'dart', 'lambda': 1.2856379394530026e-05, 'alpha': 2.7722380639012295e-06, 'max_depth': 3, 'eta': 0.03457484593308945, 'gamma': 1.2707911897752756e-05, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.026758673262209244, 'skip_drop': 0.014320827416091277}. Best is trial 58 with value: 0.8370786516853933.


FrozenTrial(number=58, values=[0.8370786516853933], datetime_start=datetime.datetime(2021, 10, 2, 20, 11, 15, 893062), datetime_complete=datetime.datetime(2021, 10, 2, 20, 11, 16, 249779), params={'booster': 'dart', 'lambda': 6.810559266757943e-05, 'alpha': 2.8673457152075684e-05, 'max_depth': 3, 'eta': 0.03879555008580796, 'gamma': 0.014916783848087345, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 9.819154679456242e-08, 'skip_drop': 0.012665326127287846}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08), 'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'eta': LogUniformDistribution(high=1.0, low=1e-08), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide')), 'sample_type': CategoricalDistribution(choices=

## Catboost

In [47]:
#optuna catboost example
#https://github.com/optuna/optuna-examples/blob/main/catboost/catboost_simple.py
import catboost as cb
def objective(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
        
        #save the five fold accuracy
        accuracy = []
        
        gbm = cb.CatBoostClassifier(**param)
        gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
        
        #save the best model for each fold 
        joblib.dump(gbm, f'catboost_fold{i}.pkl')
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-02 20:11:29,125] A new study created in memory with name: no-name-3e680299-6d31-4e87-9b26-895c121d287a
[I 2021-10-02 20:11:34,140] Trial 0 finished with value: 0.8033707865168539 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.027630804811680683, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.8033707865168539.
[I 2021-10-02 20:11:36,728] Trial 1 finished with value: 0.8146067415730337 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.05770782080345352, 'depth': 1, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.10180212725229842}. Best is trial 1 with value: 0.8146067415730337.
[I 2021-10-02 20:11:41,511] Trial 2 finished with value: 0.797752808988764 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.015033223032787178, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 1 with value: 0.8146067415730337.
[I 2021-10-02

[I 2021-10-02 20:12:38,283] Trial 25 finished with value: 0.8314606741573034 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.07712222161069392, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.2650194523932219}. Best is trial 11 with value: 0.8370786516853933.
[I 2021-10-02 20:12:40,571] Trial 26 finished with value: 0.8089887640449438 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.06427819501994521, 'depth': 12, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.2625317517593312}. Best is trial 11 with value: 0.8370786516853933.
[I 2021-10-02 20:12:44,168] Trial 27 finished with value: 0.8202247191011236 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.09157404998712938, 'depth': 11, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 11 with value: 0.8370786516853933.
[I 2021-10-02 20:12:46,410] Trial 28 finished with value: 0.8146067415730337 and parameters: 

KeyboardInterrupt: 

## Logistic and Random Forest and SVC

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = trial.suggest_categorical("classifier", ["LogReg", "SVC", "RandomForest"])
    
    # Step 2. Setup values for the hyperparameters:
    if classifier_name == 'LogReg':
        logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
        classifier_obj = linear_model.LogisticRegression(C=logreg_c)
    elif classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
    else:
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators
        )
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'log_randomf_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Logistic

In [ ]:
#logistic

#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'LogReg'
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    classifier_obj = linear_model.LogisticRegression(C=logreg_c)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'logistic_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## SVC

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    # Step 2. Setup values for the hyperparameters:
    classifier_name = "SVC"
    svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
    classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'svc_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## RandomForest

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'RandomForest'
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
#     rf_min_depth = trial.suggest_int("rf_min_depth", 2, 32, log=True)
    classifier_obj = ensemble.RandomForestClassifier(
        max_depth=rf_max_depth, n_estimators=rf_n_estimators
    )
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'rf_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Decision Tree

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 612)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 612)
    max_leaf_nodes = int(trial.suggest_int("max_leaf_nodes", 2, 612))
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    DTC = DecisionTreeClassifier(min_samples_split = min_samples_split, 
                                 max_leaf_nodes = max_leaf_nodes,
                                 criterion = criterion)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        DTC.fit(train_x, train_y)
        preds = DTC.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(DTC, f'dt_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Ensemble Model

In [ ]:
X_train = train.drop(['Survived', 'fold'], axis=1).values
y_train = train[['Survived']].values

In [ ]:
test2 = test.drop(['Survived', 'fold'], axis=1).values

In [ ]:
#get the mean of best fold in lgb   0.8338945005611672  0.77990 selected
lgb_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./lgb_fold{i}.pkl')
    lgb_pred += model.predict(test2) / 5
    
lgb_pred = lgb_pred.round().astype(int)

In [ ]:
#get the mean of best fold in xgboost  0.8314606741573034  0.78468 selected
from xgboost import XGBClassifier
xgboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./xgboost_fold{i}.pkl')
    model = XGBClassifier()
    model.fit(X_train, y_train)
    xgboost_pred += model.predict_proba(test2)[:,1] / 5   #predict probability [0, 1]
    
xgboost_pred = xgboost_pred.round().astype(int)

In [ ]:
#get the mean of best fold in catboost  0.8426966292134831  nice  0.77990 selected
catboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./catboost_fold{i}.pkl')
    catboost_pred += model.predict(test2) / 5
    
catboost_pred = catboost_pred.round().astype(int)

In [ ]:
#get the mean of best fold in logistic  0.8426966292134831 0.78947 selected nice
logistic_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./logistic_fold{i}.pkl')
    logistic_pred += model.predict(test2) / 5
    
logistic_pred = logistic_pred.round().astype(int)

In [ ]:
#get the mean of best fold in logistic/randomForest/svc  
#randomForest win 0.8314606741573034
log_ran_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./log_randomf_fold{i}.pkl')
    log_ran_pred += model.predict(test2) / 5
    
log_ran_pred = log_ran_pred.round().astype(int)

In [ ]:
#get the mean of best fold in svc  0.7528089887640449    0.66985
svc_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./svc_fold{i}.pkl')
    svc_pred += model.predict(test2) / 5
    
svc_pred = svc_pred.round().astype(int)

In [ ]:
#get the mean of best fold in randomForest   0.8370786516853933/0.8314606741573034(add min)  0.69138/0.78468(min)
rf_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./rf_fold{i}.pkl')
    rf_pred += model.predict(test2) / 5

    
rf_pred = rf_pred.round().astype(int)

In [ ]:
#get the mean of best fold in decisionTree   0.8258426966292135   0.78947 selected nice
dt_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./dt_fold{i}.pkl')
    dt_pred += model.predict(test2) / 5

    
dt_pred = dt_pred.round().astype(int)

#### way 1:

In [ ]:
#combine lgb_pred rf_pred
f_pro = (lgb_pred + xgboost_pred + catboost_pred + logistic_pred)/4
# f_pro = dt_pred

In [ ]:
pred = []
for i in f_pro:
    if i > 0.5:
        pred.append(1)
    else:
        pred.append(0)

#### way 2:

In [251]:
# pred2 = []
# for i in range(418):
#     count_0 = 0
#     count_1 = 0
#     for model in [lgb_pred, xgboost_pred, catboost_pred, dt_pred, logistic_pred]:
#         if model[i] == 0:
#             count_0 += 1
#         else:
#             count_1 += 1
            
#     if count_0 >= count_1:
#         pred2.append(0)
#     else:
#         pred2.append(1)
# # print(pred2)   

In [ ]:
sub.Survived = pred

In [ ]:
sub

In [355]:
sub.to_csv('submission_mean_model4.csv', index=False)